## Исследование надёжности заёмщиков

Задача — разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Данные — статистика о платёжеспособности клиентов. Включают в себя основную информацию о клиенте, такую как:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Результаты исследования могут быть учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Предобработка данных

In [17]:
import pandas as pd
data = pd.read_csv('/Users/kirillshaplyko1/Yandex.Disk.localized/Обучение/Яндекс практикум/Проекты/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В столбце days_employed отрицательные числа, в столбце education встречаются названия с разным регистром

**Обработка пропусков**

In [19]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения встречаются в столбцах days_employed и total_income. Это может быть связано с отсутствием работы у заемщика.

**Замена типа данных**

In [20]:
income_median = data['total_income'].median()
data = data.fillna(income_median)
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заполнил пропуски медианой по доходам

**Обработка дубликатов**

In [21]:
#data.duplicated().sum()
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

Изначально было 54 дубликата. Удалил дубликаты и сбросил значения индекса.

### Лемматизация

Нужна категоризация по цели кредита. Текущие значения в столбце purpose неоднородные. Для этого вначале проведем лемматизацию - разобьем каждую ячейку на отдельные слова и по ключевым словам присвоим категорию.

In [22]:
#print(data['purpose'].duplicated().sum())
purpose = data['purpose'].drop_duplicates().reset_index(drop = True)
from pymystem3 import Mystem
m = Mystem()
def purpose_group(purpose):
    lemma = m.lemmatize(purpose)
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'недвижимость' in lemma:
        return 'недвижимость'
    if 'жилье' in lemma:
        return 'недвижимость'
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'образование' in lemma:
        return 'образование'
    return 'ошибка'
data['purpose_group'] = data['purpose'].apply(purpose_group)
data.head(2)
#print(data[data['purpose_group']=='ошибка'].count())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль


Сделал лемматизацию, присвоил каждому значение в столбце цель кредита уникальное значение, добавил новый столбец для этого. Проверил, что ошибок нет.

### Категоризация данных

In [23]:
purpose_group_dict = data['purpose_group'].unique()
median_income = (data['total_income'].median())
def children_group(children):
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'
data['children_group'] = data['children'].apply(children_group)
def family_status_group(family_status):
    if family_status == 'женат / замужем':
        return 'в браке'
    if family_status == 'гражданский брак':
        return 'гражданский брак'
    if family_status == 'вдовец / вдова':
        return 'вдовство'
    if family_status == 'в разводе':
        return 'развод'
    return 'не в браке'
data['family_status_group'] = data['family_status'].apply(family_status_group)

def income_group(total_income):
    if total_income < median_income/3*2:
        return 'ниже среднего'
    elif median_income/3*2 < total_income < median_income/3*4:
        return 'средний'
    else:
        return 'выше среднего'
data['income_group'] = data['total_income'].apply(income_group)
data.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,children_group,family_status_group,income_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,есть дети,в браке,выше среднего
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,есть дети,в браке,средний


### Добавил категории: 
1. Есть или нет детей
2. Семейный статус. Оставил категории, которые присутствовали в данных - в браке, гражданский брак, вдовство, развод и не в браке. Некоторые сократил по названию.
3. Группы по уровню дохода разбил следующим образом: посчитал медиану. Группа ниже среднего - все значения, которые на 1/3 меньше медианного; группа выше среднего - все значения, которые на 1/3 больше медианного; остальные - средний доход. Такую разбивку делал интуитивно, исходя из цифры медианы и собственного опыта.

### Исследовательский анализ

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [24]:
children_pivot = data.pivot_table(index='children_group',columns = 'debt',values='children',aggfunc='count')
children_pivot.columns = ['выплаты','просрочка']
children_pivot = children_pivot.transpose()
children_pivot['%, есть дети']=children_pivot['есть дети']/len(data[data['children_group']=='есть дети'])*100
children_pivot['%, нет детей']=children_pivot['нет детей']/len(data[data['children_group']=='нет детей'])*100
print(children_pivot)

children_group  есть дети  нет детей  %, есть дети  %, нет детей
выплаты              6640      13090     90.747574      92.48269
просрочка             677       1064      9.252426       7.51731


**Вывод:** заемщики без детей платят чуть лучше, чем заемщики с детьми - в 92.4% случаев против 90.7%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
income_pivot = data.pivot_table(index='income_group',columns='debt',values='family_status',aggfunc='count')
income_pivot.columns = ['выплаты','задолженность']
income_pivot['общее']=income_pivot['выплаты']+income_pivot['задолженность']
income_pivot['% выплат']=income_pivot['выплаты']/income_pivot['общее']*100
income_pivot['% задолженности']=income_pivot['задолженность']/income_pivot['общее']*100
print(income_pivot.sort_values('% выплат',ascending=False))

               выплаты  задолженность  общее   % выплат  % задолженности
income_group                                                            
выше среднего     5123            396   5519  92.824787         7.175213
ниже среднего     3748            326   4074  91.998036         8.001964
средний          10859           1019  11878  91.421115         8.578885


**Вывод:** лучше всего платят люди с высоким уровнем дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
purpose_pivot = data.pivot_table(index='purpose_group',columns='debt',values='family_status',aggfunc='count')
#print(purpose_pivot.head())
purpose_pivot.columns = ['выплаты','задолженность']
purpose_pivot['общее']=purpose_pivot['выплаты']+purpose_pivot['задолженность']
purpose_pivot['% выплат']=purpose_pivot['выплаты']/purpose_pivot['общее']*100
purpose_pivot['% задолженности']=purpose_pivot['задолженность']/purpose_pivot['общее']*100
print(purpose_pivot.sort_values('% выплат',ascending=False))

               выплаты  задолженность  общее   % выплат  % задолженности
purpose_group                                                           
недвижимость     10032            782  10814  92.768633         7.231367
свадьба           2149            186   2335  92.034261         7.965739
образование       3644            370   4014  90.782262         9.217738
автомобиль        3905            403   4308  90.645311         9.354689


**Вывод:** больше всего заемщики берут кредиты на недвижимость. Следовательно и объемы вылат и просрочки выше всего по этим категориям. 

Больше всего кредитов действительно взято на недвижимость, банк и зарабатывает на этих клиентах больше всего, и долгов по общему объему в этой категории конечно же больше, просто потому что тут крутятся большие деньги. Но если посмотреть на процент задолженности, то видно, что человек, который берет кредит на автомобиль в 9,35% случаев его не вернет, а тот, кто берет кредит на недвижимость, не вернет его только в 7,2% случаев. Так как банк хочет вернуть свои деньги назад, выгодней выдавать кредит тому, у кого меньше вероятность задолженности.

### Общий вывод. 
Чаще всего люди берут кредиты на недвижимость. Они так же допускают меньше всего просрочки по платежам.
Что касается наличие детей - тут ситуация лучше у бездетных, 92.4% случаев против 90.7%